In [ ]:
from pathlib import Path
import pandas as pd
import plotly.express as px
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
from datetime import datetime
import scipy.stats as scs

# 1. Loading the data

In [ ]:
DATA_PATH = Path("data")
WEATHER_DATA_PATH = DATA_PATH / "RR59"

In [ ]:
weather_data = pd.read_csv(WEATHER_DATA_PATH / "Q_59_previous-1950-2022_RR-T-Vent.csv", sep = ";", parse_dates=["AAAAMMJJ"])

## 1.1 Selecting one station of interest

In [ ]:
station_of_interest = "DUNKERQUE"

In [ ]:
dunkerque_data = weather_data[weather_data["NOM_USUEL"] == station_of_interest]
dunkerque_rainfall = dunkerque_data.set_index("AAAAMMJJ")["RR"]

# 2. Time serie analysis

In [ ]:
# TODO : Plot timeserie

In [ ]:
# TODO : Compute timeserie autocorrelation

# 3. Study yearly maximum

## 3.1 Resampling data

In [ ]:
# TODO : resample data to only have yearly maximum

In [ ]:
# TODO : plot histogram

## 3.2 Try different statistical fit on data

In [ ]:
# TODO : Try different fit presented in the course, using scipy.stats

## 3.3 Compare Exceedance probabilities of different distributions

In [ ]:
# TODO : Compare exceedance probabilities for each of your model

# 4. Numerical approximations

In [ ]:
# TODO : Calibrate gumbel distribution using two different scipy method (see documentation)

In [ ]:
# TODO : Compare extreme events probabilities of both gumbel models

# 5. Pyextremes pipeline

## 5.1 Block Maxima

In [ ]:
from pyextremes import EVA

model_bm = EVA(dunkerque_rainfall)

In [ ]:
# Choose one modelling of extremes using Block Maxima with different input parameters

In [ ]:
# Compute diagnosis and comment

## 5.2 POT

In [ ]:
model_pot = EVA(dunkerque_rainfall)

In [ ]:
# Choose one modelling of extremes using Peak over Threshold with different input parameters

In [ ]:
# Compute diagnosis and comment

What is the return period (in years) of a daily rainfall superior than 80cm at Dunkerque ?

In [ ]:
# Answer the question

# 6. (Optional) Seasonality and trend

In [ ]:
# TODO : Compute time serie seasonality and trend

In [ ]:
# TODO : Remove seasonality and trend from the data

In [ ]:
# TODO : Perform the EVA analysis

In [ ]:
# TODO : Add the seasonality and trend to the simulated data. Do you change your opinion on the notebook question ?